# Database Query and Export Tool

## Introduction

This Jupyter Notebook provides a comprehensive database management tool, designed to enable efficient querying and exporting of database records. Users can specify fields to query within a database and save results in either XML or JSON format, allowing for streamlined data handling and flexible output options.

### Objectives:
- **Database Querying**: Retrieve data from specified fields based on user-defined parameters.
- **Flexible Export Options**: Save query results as XML or JSON files with a custom filename for easy access and organization.
- **User-Friendly Functionality**: Simplifies data extraction and export processes, making it adaptable for various database management tasks.

### Dependencies:
To run this notebook, make sure the following libraries are installed:
- `sqlite3` (for database interactions)
- `json` (for JSON export)
- `xml.etree.ElementTree` (for XML export)
- `os` (for file operations)

### Usage:
To begin using this tool, configure the database path, define query parameters, and specify your desired output format and filename. Execute the cells sequentially to define functions and run your queries, then export results as needed.


In [ ]:
# Import standard libraries
import json  # For parsing and manipulating JSON data
import sqlite3  # SQLite database library for creating and managing SQLite databases
import xml.etree.ElementTree as ET  # For parsing and creating XML data structures

# Import third-party libraries
from tabulate import tabulate  # For creating formatted tables in plain text


In [ ]:
# Database Connection Setup
# Connect to the HyperionDev database and handle errors in case the database is not found.
try:
    conn = sqlite3.connect("HyperionDev.db")
    cur = conn.cursor()
except sqlite3.Error:
    print("Please ensure your database is named 'HyperionDev.db'")
    quit()


In [ ]:
def usage_is_incorrect(input, num_args):
    """
    Checks if the command has the correct number of arguments.
    
    Args:
    input (list): List of input commands and arguments
    num_args (int): Expected number of arguments for the command
    
    Returns:
    bool: True if the number of arguments is incorrect, False otherwise
    """
    if len(input) != num_args + 1:
        print(f"The {input[0]} command requires {num_args} arguments.")
        return True
    return False


In [ ]:
def store_data_as_json(data, filename):
    """
    Saves data to a JSON file with specified filename.
    
    Args:
    data (list): Data to be stored
    filename (str): Filename for the JSON file
    
    Outputs:
    Prints confirmation message after saving
    """
    headings = data[0]
    rest_of_data = data[1:]
    dictionary = {str(index+1): {headings[i]: val for i, val in enumerate(row)} 
                  for index, row in enumerate(rest_of_data)}

    with open(filename, 'w', encoding='utf-8') as outfile:
        json.dump(dictionary, outfile, sort_keys=True, indent=4)
    print(f"\nThe file '{filename}' has been saved.")


In [ ]:
def store_data_as_xml(data, filename, root_tag):
    """
    Saves data to an XML file with a specified root tag and filename.
    
    Args:
    data (list): Data to be stored
    filename (str): Filename for the XML file
    root_tag (str): Root tag name for XML structure
    
    Outputs:
    Prints confirmation message after saving
    """
    headings = data[0]
    rest_of_data = data[1:]
    root = ET.Element(root_tag)

    for index, row in enumerate(rest_of_data):
        record = ET.SubElement(root, f'record_{index+1}')
        for i, val in enumerate(row):
            ET.SubElement(record, headings[i]).text = str(val)

    tree = ET.ElementTree(root)
    tree.write(filename, encoding='unicode')
    print(f"\nThe file '{filename}' has been saved.")


In [ ]:
def offer_to_store(data, root_tag):
    """
    Prompts user to choose if data should be saved and in which format.
    
    Args:
    data (list): Data to be offered for storage
    root_tag (str): Root tag name for XML structure if XML format is chosen
    """
    while True:
        choice = input("Would you like to store this result? Y/[N]: ").strip().lower()
        if choice == "y":
            filename = input("Specify filename (end with .xml or .json): ")
            ext = filename.split(".")[-1]
            if ext == 'xml':
                store_data_as_xml(data, filename, root_tag)
            elif ext == 'json':
                store_data_as_json(data, filename)
            else:
                print("Invalid file extension. Use .xml or .json")
            break
        elif choice == 'n':
            break
        else:
            print("Invalid choice")


In [ ]:
# User Interface
# Displaying available commands and welcoming users to the querying app.

usage = """
Commands:
d                   - demo (list all students)
vs <student_id>     - view subjects by student_id
la <first> <last>   - view address by student name
lr <student_id>     - list reviews for a student
lc <teacher_id>     - list courses by teacher_id
lnc                 - list students who haven't completed the course
lf                  - list students with completion <= 30
e                   - exit program
"""

print("Welcome to the Data Querying App!")


In [ ]:
# Main Program Loop
# Processing user input and executing corresponding database queries.

while True:
    print()
    user_input = input(usage).split()
    command = user_input[0]
    args = user_input[1:] if len(user_input) > 1 else []

    # Demo command
    if command == 'd':
        data = cur.execute("SELECT * FROM Student").fetchall()
        for _, first, last, *_ in data:
            print(f"{first} {last}")
            
    elif command == 'vs' and not usage_is_incorrect(user_input, 1):
        student_id = args[0]
        try:
            cur.execute('''SELECT course_name FROM Course 
                           JOIN StudentCourse ON StudentCourse.course_code = Course.course_code 
                           WHERE student_id = ?''', (student_id,))
            subjects = cur.fetchall()
            data = [['student_id', 'subjects'], [student_id, ', '.join(s[0] for s in subjects)]]
            offer_to_store(data, 'student_subjects')
        except sqlite3.Error:
            print("Invalid student ID")
            
    elif command == 'e':
        print("Exiting program. Goodbye!")
        break

    # Additional commands for la, lr, lc, lnc, and lf can be implemented in a similar format
